# Results Analysis: LCIA QSAR Modeling Framework
**Date:** August, 31, 2023 <br>
**Author:** Jacob Kvasnicka

In [1]:
import matplotlib
# matplotlib.use('Agg')  # avoids rendering figures
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from raw_processing import other_sources
from config_management import UnifiedConfiguration
from data_management import DataManager
from metrics_management import MetricsManager
from results_management import ResultsManager
from results_analysis import ResultsAnalyzer
from plotting.moe import MOE_CATEGORIES
from plotting import sensitivity_analysis

config_mapping_path = 'Input\Configuration\configuration-mapping.json'
config = UnifiedConfiguration(config_mapping_path)

data_manager = DataManager(config.data, config.path)
metrics_manager = MetricsManager(config.category_to_dict('metric'))
results_manager = ResultsManager(
    output_dir='Results',
    results_file_type=config.data.file_type
)
results_analyzer = ResultsAnalyzer(
    results_manager, 
    data_manager,
    config.plot
)

key_for_effect = {
    'general' : {
      "target_effect" : "general",
      "features_source" : "opera",
      "ld50_type" : "predicted",
      "data_condition" : "missing",
      "select_features" : "true",
      "estimators" : "RandomForestRegressor"
    },

    'repro_dev' : {
      "target_effect" : "repro_dev",
      "features_source" : "opera",
      "ld50_type" : "predicted",
      "data_condition" : "missing",
      "select_features" : "true",
      "estimators" : "RandomForestRegressor"
    }
}

def get_model_key(effect):
    return tuple(key_for_effect[effect].values())

percentiles = [0.05, 0.5, 0.95]

## Dataset Characterization

In [2]:
raw_surrogate_pods = other_sources.toxicity_data_and_study_counts_from_excel(
    config.path.raw_surrogate_pods_file, 
    config.raw_data.tox_metric, 
    'dtxsid', 
    config.raw_data.surrogate_tox_data_kwargs
)

for effect in list(raw_surrogate_pods.columns.unique(level=0)):
    N = len(raw_surrogate_pods[effect].dropna())
    print(f'{N} chemicals for {effect}')

5209 chemicals for non-reproductive/developmental effects
4938 chemicals for reproductive/developmental effects


In [3]:
surrogate_pods = pd.read_csv(
    config.path.surrogate_pods_file, 
    index_col=0
)

for effect, y in list(surrogate_pods.items()):
    N = len(y.dropna())
    print(f'{N} chemicals for {effect}')

2404 chemicals for general
2999 chemicals for repro_dev


In [4]:
for effect in surrogate_pods:    
    _, y = results_analyzer.load_features_and_target(**key_for_effect[effect])
    N = len(y)
    print(f'{N} chemicals for {effect}')    

1791 chemicals for general
2228 chemicals for repro_dev


In [5]:
application_chemicals = data_manager.load_application_chemicals()

print(f'{len(application_chemicals)} application chemicals')

32524 application chemicals


In [6]:
def percents_missing(X):
    '''
    Compute the percentages of samples with complete data for each feature.
    '''
    return ((X.isna().sum() / len(X)).sort_values(ascending=False).round(2) * 100).head()

In [7]:
features_path = config.path.file_for_features_source['opera']

X = pd.read_parquet(features_path)

percents_missing(X)

BioDeg_HalfLife_pred    72.0
CACO2_pred              45.0
OH_pred                 37.0
Clint_pred              22.0
KM_pred                 21.0
dtype: float64

In [8]:
for effect in key_for_effect:
    
    print(effect)
    
    X, y = results_analyzer.load_features_and_target(**key_for_effect[effect])
    
    print(percents_missing(X))

general
BioDeg_HalfLife_pred    74.0
CACO2_pred              49.0
OH_pred                 44.0
KOA_pred                23.0
KM_pred                 23.0
dtype: float64
repro_dev
BioDeg_HalfLife_pred    75.0
CACO2_pred              49.0
OH_pred                 45.0
KM_pred                 25.0
Clint_pred              24.0
dtype: float64


## Model Evaluation & Important Features

In [9]:
def describe_result(effect, result_type):
    '''
    Return the summary statistics with confidence interval.
    '''    
    if 'importances' in result_type:
        metrics = list(config.plot.label_for_scoring)
    else: 
        metrics = list(config.plot.label_for_metric)
        other_metric = 'mean_absolute_error'
        if other_metric not in metrics:
            metrics.append(other_metric)
            
    if 'root_mean_squared_error' in metrics:
        metrics.append('gsd')
        metrics.append('gsd_squared')
        # Compute GSD squared.     
        performances = results_analyzer.read_result(get_model_key(effect), result_type)
        rmse = performances['root_mean_squared_error']
        gsd = 10**rmse  # in natural units
        performances['gsd'] = gsd
        performances['gsd_squared'] = gsd**1.96

    desc = performances.describe(percentiles=percentiles)[metrics].round(2)
    
    return desc.loc[[k for k in desc.index if '%' in k]].T

In [10]:
describe_result('general', 'performances')

,5%,50%,95%
metric,,,
root_mean_squared_error,0.64,0.69,0.76
median_absolute_error,0.37,0.40,0.44
r2_score,0.41,0.48,0.53
mean_absolute_error,0.49,0.52,0.55
gsd,4.37,4.91,5.70
gsd_squared,18.01,22.63,30.29


In [11]:
describe_result('repro_dev', 'performances')

,5%,50%,95%
metric,,,
root_mean_squared_error,0.54,0.58,0.72
median_absolute_error,0.28,0.31,0.34
r2_score,0.38,0.49,0.56
mean_absolute_error,0.39,0.42,0.45
gsd,3.46,3.81,5.26
gsd_squared,11.39,13.76,25.89


In [12]:
important_features = {}

for effect in key_for_effect:
    
    important_features[effect] = set(results_analyzer.get_important_features(get_model_key(effect)))

In [13]:
important_features['general'].intersection(important_features['repro_dev'])

{'CATMoS_LD50_pred',
 'CombDipolPolariz',
 'ReadyBiodeg_pred_discrete',
 'WS_pred'}

In [14]:
important_features['general'].difference(important_features['repro_dev'])

{'MP_pred',
 'P_pred',
 'TopoPolSurfAir',
 'VP_pred',
 'nbN_discrete',
 'ndHBdDon_discrete'}

In [15]:
important_features['repro_dev'].difference(important_features['general'])

{'BCF_pred', 'FUB_pred', 'KM_pred', 'KOA_pred', 'Koc_pred', 'Sp3Sp2HybRatio'}

## Model Application

### POD cumulative frequencies

In [16]:
pod_data = {
    effect : results_analyzer.pod_and_prediction_interval(
        get_model_key(effect),
        inverse_transform=True, 
        normalize=True
    )
    for effect in key_for_effect
}

# Get median value with prediction interval
for effect, df in pod_data.items():
    median = df['pod'].quantile()
    # Find the closest index to the median to avoid float issues
    abs_diff = (df['pod'] - median).abs()
    idx = abs_diff.idxmin()
    median_data = df.loc[idx]
    print(f"{effect}: {median} ({median_data['lb']}, {median_data['ub']})")

general: 11.218834721209232 (0.818494954674385, 153.77278965864915)
repro_dev: 31.08881947251823 (3.442790465052367, 280.7366822905897)


### Compare POD estimates (benchmarking)

In [17]:
# NOTE: Not sure whether these data are used in the paper
pod_comparison_data = {
    effect : results_analyzer.get_pod_comparison_data(get_model_key(effect)) 
    for effect in key_for_effect
}

# Inverse transform the PODs from log10
(10**pd.DataFrame(pod_comparison_data['general']).quantile(percentiles)).round(1)

,Authoritative,ToxValDB Surrogate,QSAR
0.05,0.0,0.3,2.0
0.50,4.4,12.8,11.3
0.95,212.8,268.6,93.9


In [18]:
(10**pd.DataFrame(pod_comparison_data['repro_dev']).quantile(percentiles)).round(1)

,Authoritative,ToxValDB Surrogate,QSAR
0.05,0.1,1.1,4.9
0.50,4.3,49.0,31.7
0.95,93.1,331.8,176.6


## Sensitivity Analysis

In [2]:
def summarize_model_performances(
        results_manager, 
        data_manager,
        plot_settings,
        model_keys=None, 
        quantiles=None
        ):
    '''
    Generate a statistical summary table comparing performance scores of each 
    model.

    Parameters
    ----------
    results_manager : instance of ResultsManager
        Object responsible for managing and combining results from models.
    data_manager : instance of DataManager
        Data manager object for loading dataset features and targets.
    plot_settings : SimpleNamespace
        Contains settings for plotting, such as model labels, metrics, and effects.
    model_keys : list of tuple, optional
        List of model keys for which to retrieve the results. If None, 
        then all model keys will be used.
    quantiles : float or array-like, optional
        Value between 0 <= q <= 1, the quantile(s) to compute. Defaults to 90%
        and 95% confidence intervals.
        
    Returns
    -------
    pandas.DataFrame
        The index has three levels: effect, model_name, metric. The columns are
        the quantiles.
    '''
    if not quantiles:
        quantiles = [0.025, 0.05, 0.5, 0.95, 0.975]  # by default

    performances = results_manager.combine_results(
        'performances', 
        model_keys=model_keys
    )
    # Get the results for the metrics of interest
    metrics = list(plot_settings.label_for_metric)
    performances = performances.loc[
        :, performances.columns.get_level_values('metric').isin(metrics)
    ]
    # Use the nice labels in the config file
    performances = performances.rename(
        plot_settings.label_for_metric, 
        level='metric', 
        axis=1
    )

    model_key_names = results_manager.read_model_key_names()

    performances_for = {}  # initialize

    for effect, effect_label in plot_settings.label_for_effect.items():
        performances_for[effect_label] = sensitivity_analysis.prepare_data_for_plotting(
            performances, 
            effect, 
            data_manager, 
            model_key_names, 
            plot_settings
        )

    # Create a statistical summary table
    performance_summary = (
        pd.concat(performances_for, axis=1)
        .quantile(quantiles)
        .T
    )
    # Name the first index level
    performance_summary.index.names = ['effect'] + performance_summary.index.names[1:]

    return performance_summary

In [3]:
keys_without_selection = results_manager.read_model_keys(
    inclusion_string='false'
    )

summary_without_selection = summarize_model_performances(
    results_manager, 
    data_manager,
    config.plot,
    model_keys=keys_without_selection
)

summary_without_selection

0.025  \
effect                     model_name                        metric             
General Noncancer          RandomForestRegressor (1,791)     RMSE    0.617446   
                                                             MedAE   0.345249   
                                                             $R^2$   0.427616   
                           GradientBoostingRegressor (1,791) RMSE    0.630407   
                                                             MedAE   0.367059   
...                                                                       ...   
Reproductive/Developmental RDKit Features (2,224)            MedAE   0.290147   
                                                             $R^2$   0.363714   
                           No Imputation (227)               RMSE    0.339944   
                                                             MedAE   0.180939   
                                                             $R^2$   0.184882   

                                                                        0.050  \
effect                     model_name                        metric             
General Noncancer          RandomForestRegressor (1,791)     RMSE    0.624314   
                                                             MedAE   0.352830   
                                                             $R^2$   0.440533   
                           GradientBoostingRegressor (1,791) RMSE    0.637902   
                                                             MedAE   0.369946   
...                                                                       ...   
Reproductive/Developmental RDKit Features (2,224)            MedAE   0.292596   
                                                             $R^2$   0.374364   
                           No Imputation (227)               RMSE    0.352791   
                                                             MedAE   0.201942   
                                                             $R^2$   0.214574   

                                                                        0.500  \
effect                     model_name                        metric             
General Noncancer          RandomForestRegressor (1,791)     RMSE    0.676581   
                                                             MedAE   0.391404   
                                                             $R^2$   0.498994   
                           GradientBoostingRegressor (1,791) RMSE    0.691876   
                                                             MedAE   0.413094   
...                                                                       ...   
Reproductive/Developmental RDKit Features (2,224)            MedAE   0.318438   
                                                             $R^2$   0.447400   
                           No Imputation (227)               RMSE    0.453375   
                                                             MedAE   0.276368   
                                                             $R^2$   0.397379   

                                                                        0.950  \
effect                     model_name                        metric             
General Noncancer          RandomForestRegressor (1,791)     RMSE    0.744249   
                                                             MedAE   0.425172   
                                                             $R^2$   0.556304   
                           GradientBoostingRegressor (1,791) RMSE    0.747987   
                                                             MedAE   0.457292   
...                                                                       ...   
Reproductive/Developmental RDKit Features (2,224)            MedAE   0.346989   
                                                             $R^2$   0.524548   
                           No Imputation (227)               RMSE    0.545985   
                   

In [4]:
keys_with_selection = results_manager.read_model_keys(
    inclusion_string='true'
    )

summary_with_selection = summarize_model_performances(
    results_manager, 
    data_manager,
    config.plot,
    model_keys=keys_with_selection
)

summary_with_selection

0.025  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.631001   
                                                                 MedAE   0.359110   
                                                                 $R^2$   0.402050   
                                                                 RMSE    0.631001   
                                                                 MedAE   0.359110   
                                                                 $R^2$   0.402050   
Reproductive/Developmental RandomForestRegressor (final) (2,228) RMSE    0.523493   
                                                                 MedAE   0.275471   
                                                                 $R^2$   0.363602   
                                                                 RMSE    0.523493   
                                                                 MedAE   0.275471   
                                                                 $R^2$   0.363602   

                                                                            0.050  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.640619   
                                                                 MedAE   0.366414   
                                                                 $R^2$   0.408917   
                                                                 RMSE    0.640619   
                                                                 MedAE   0.366414   
                                                                 $R^2$   0.408917   
Reproductive/Developmental RandomForestRegressor (final) (2,228) RMSE    0.539058   
                                                                 MedAE   0.284368   
                                                                 $R^2$   0.381948   
                                                                 RMSE    0.539058   
                                                                 MedAE   0.284368   
                                                                 $R^2$   0.381948   

                                                                            0.500  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.691144   
                                                                 MedAE   0.402057   
                                                                 $R^2$   0.479789   
                                                                 RMSE    0.691144   
                                                                 MedAE   0.402057   
                                                                 $R^2$   0.479789   
Reproductive/Developmental RandomForestRegressor (final) (2,228) RMSE    0.580969   
                                                                 MedAE   0.312614   
                                                                 $R^2$   0.494801   
                                                                 RMSE    0.580969   
                                                                 MedAE   0.312614   
                                                                 $R^2$   0.494801   

                                                                            0.950  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.755731   
                                                                 MedAE   0.437987   
                                                                 $R^2$   0.532331   
                                                                 RMSE    0.755

### Margins of exposure with cumulative counts

In [23]:
# Get the upper bounds of the MOE concern categories
thres_for_concern = {
    k.lower().replace(' ', '_') : bounds[-1] 
    for k, bounds in MOE_CATEGORIES.items()
}

for effect in key_for_effect:
    print(f'For {effect} effects:')

    results_for_percentile = results_analyzer.moe_and_prediction_intervals(get_model_key(effect))

    # Get the upper bound of exposure uncertainty
    ub_exposure_results = results_for_percentile['95th percentile (mg/kg/day)']
    # Get MOEs for the lower bound of the POD prediction interval
    moes = ub_exposure_results['lb']

    for concern, concern_threshold in thres_for_concern.items():
        where_potential_concern = moes <= concern_threshold
        concern_count = ub_exposure_results.loc[where_potential_concern]['cum_count'][-1]
        print(f'\t{concern_count} chemicals {concern}')

For general effects:
	33277 chemicals low_concern
	2337 chemicals moderate_concern
	497 chemicals high_concern
For repro_dev effects:
	32858 chemicals low_concern
	1458 chemicals moderate_concern
	189 chemicals high_concern


In [24]:
exposure_df = data_manager.load_exposure_data().loc[application_chemicals]

exposure_difference = (
    exposure_df['95th percentile (mg/kg/day)'] 
    - exposure_df['5th percentile (mg/kg/day)']
)

typical_uncertainty = exposure_difference.median()

print(f'Typical exposure uncertainty: {round(typical_uncertainty)} log10-units')

Typical exposure uncertainty: 4 log10-units
